# Исследование надёжности заёмщиков

Для кредитного отдела банка на основе статистика о платёжеспособности клиентов необходимо выяснить, как влияют на факт погашения кредита в срок различные данные клиента.

## Общая информация

In [1]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')
print(df.head())
print(df.tail())


   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


21525 строк
типы данных:float64(2), int64(5), object(5)
в 2 строках total_income и days_employed пропуски около 10 %

Посмотрим на уникальные значения в каждом столбце:

In [3]:
for row in df: 
      print(df[row].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

переименуем столбец со стажем, чтобы мне было проще ориентироваться в значения потом

In [4]:
df.set_axis(['children', 'years_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'], axis = 'columns', inplace = True)
df.head(3)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


**Вывод**

Всего 21525 строк. Каждая строка таблицы содержит информацию о заемщике.



 children — количество детей в семье   - целые числа, значение есть в каждой строке, есть небольшое количество отрицательных значений  
days_employed — трудовой стаж в днях  - данных нет примерно в 10% строк, есть отрицательные значения и я значения явно превышающие возможную продолжительнсоть жизни человека  
dob_days — возраст клиента в годах    - целые числа, значение есть в каждой строке, 101 значение - 0  
education — образование клиента       - значение есть в каждой строке, необходимо будет удалить дубликаты  
education_id — идентификатор образования - значение есть в каждой строке  
family_status — семейное положение    - значение есть в каждой строке  
family_status_id — идентификатор семейного положения - значение есть в каждой строке  
gender — пол клиента                  - значение есть в каждой строке  
income_type — тип занятости           - значение есть в каждой строке, представлен как object, есть редкие значения  
debt — имел ли задолженность по возврату кредитов - значение есть в каждой строке  
total_income — доход в месяц          - данных нет примерно в 10% строк, float  
purpose — цель получения кредита      - значение есть в каждой строке, много дубликатов  

Две проблемы, которые нужно решать: пропуски и отрицательные значения в трудовом стаже. Для проверки рабочих гипотез особенно ценны столбцы children, family_status и family_status_id. Данные из столбца debt позволят узнать влияние семеного положения и наличия детей на факт погашения кредита в срок.


## Предобработка данных

### Обработка пропусков

Попробуем выяснить причину неверных значений в стаже:

In [5]:
df.isnull().sum()  # <суммарное количество пропусков, выявленных методом isnull() в таблице df>
bool_series = pd.isnull(df['years_employed'])   # создадим список строк с отсутствующим значением стажа
nonemployed = df[bool_series]                  # получим таблицу состоящую из исходной с остутсвующим значением стажа
print(nonemployed.head())                      # просмотрим эту таблицу
print(nonemployed.tail())
nonemployed.info()  


    children  years_employed  dob_years education  education_id  \
12         0             NaN         65   среднее             1   
26         0             NaN         41   среднее             1   
29         0             NaN         63   среднее             1   
41         0             NaN         50   среднее             1   
55         0             NaN         54   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    пенсионер     0   
41        женат / замужем                 0      F  госслужащий     0   
55       гражданский брак                 1      F    пенсионер     1   

    total_income                           purpose  
12           NaN                   сыграть свадьбу  
26           NaN                       образование  

Никакой явной причины пропусков не вижу, предположу, что техническая ошибка, либо доход и стаж в этих анкетах не заполнялся а кредит был выдан по какому либо специальномц предложению исходя из данных уже указанных в карточке клиента

Изучу подробнее строки со стажем отрицательным и положительным:

In [6]:
print(df.loc[df.loc[:,'years_employed'] < -25000])
print(df.loc[(df.loc[:,'years_employed'] < 25000) & (df.loc[:,'years_employed'] > 0)])
print(df.loc[df.loc[:,'years_employed'] > 25000])
print(df.loc[(df.loc[:,'years_employed'] > 25000) & (df.loc[:,'income_type'] != 'пенсионер')]) # мы выяснили что положительный сверх большой стаж у пенсинеров и 2 безработных(которых всего 2)
print(400281/(2018-1953-18)/365)


Empty DataFrame
Columns: [children, years_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
Empty DataFrame
Columns: [children, years_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
       children  years_employed  dob_years education  education_id  \
4             0   340266.072047         53   среднее             1   
18            0   400281.136913         53   среднее             1   
24            1   338551.952911         57   среднее             1   
25            0   363548.489348         67   среднее             1   
30            1   335581.668515         62   среднее             1   
...         ...             ...        ...       ...           ...   
21505         0   338904.866406         53   среднее             1   
21508         0   386497.714078         62   среднее             1   
21509   

Мы увидели, что только в cтроках с отсутствием стажа нет дохода
Положительный сверх большой стаж у пенсинеров и 2 безработных(которых всего 2)
Также я предположил, что сверхбольшой стаж выражен в часах (24 часа в сутках) и увидел, что при этой гипотезе получаются адекватные значения стажа для пожилых людей на пенсии

In [7]:
df.loc[df['years_employed'] > 0, 'years_employed'] /= 8766
df.loc[df['years_employed'] < 0, 'years_employed'] /= -365.25
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   years_employed    19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
       children  years_employed  dob_years education  education_id  \
0             1       23.101090         42    высшее             0   
1             1       11.019312

In [8]:
df.groupby('dob_years')['years_employed'].mean()
df['years_employed'].mean()

12.70812094779098

Срединй стаж работы у людей с указанным возрастом  совпадает со средним стажем по всей выборке

In [9]:
df.groupby('dob_years')['years_employed'].agg(['median','mean','count'])    
print(df['dob_years'].mean())

43.29337979094077


In [10]:
df.loc[df['dob_years'] == 0, 'dob_years'] += 43
df.groupby('dob_years')['years_employed'].mean()

dob_years
19     1.734916
20     1.875275
21     1.942343
22     2.356460
23     2.265050
24     2.810145
25     2.979894
26     3.499112
27     3.965126
28     3.907649
29     4.254136
30     4.643503
31     4.593539
32     4.967713
33     5.254198
34     5.648024
35     5.837531
36     6.439835
37     6.328661
38     6.835912
39     6.867932
40     6.881438
41     7.045535
42     8.218293
43     8.331331
44     8.372781
45     8.483714
46     8.904532
47     9.157781
48     9.365563
49    10.636980
50    12.814026
51    14.341237
52    16.119717
53    17.161747
54    19.605051
55    22.906884
56    22.893403
57    26.380531
58    25.683933
59    29.811149
60    32.788695
61    31.585146
62    33.300215
63    35.769876
64    35.097721
65    35.117321
66    37.282781
67    37.427455
68    38.618322
69    39.342340
70    37.864553
71    38.646237
72    39.684790
73    37.088245
74    31.201298
75     4.596769
Name: years_employed, dtype: float64

Заменили 0 значения возраста на среднее, сразу проверили, что это не влияет кардинально на распределение стажа по возрастам.
При том что значение 0 было лишь в 0,5% строк, этот метод оправдан.

Необходимо заполнить пропуски в стаже исходя из возраста, для этого разобьем всех на несколько возрастных групп:

In [11]:
def age_group(age):
    if age <= 35:
        return 'до 35'
    elif 35 < age <= 45:
        return '35-45'
    elif 45 < age <= 55:
        return '45-55'
    else:
        return '55+'
    
df['age_group'] = df['dob_years'].apply(age_group)
df.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,23.101090,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,35-45
1,1,11.019312,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,35-45
2,0,15.396092,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,до 35
3,3,11.292942,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,до 35
4,0,38.816572,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,45-55


In [12]:
mean_stazh = df.groupby('age_group')['years_employed'].mean()
print(mean_stazh)

age_group
35-45     7.375558
45-55    13.924247
55+      31.193530
до 35     4.320118
Name: years_employed, dtype: float64


In [13]:
df.loc[(df['years_employed'].isnull()) & (df['age_group'] == 'до 35'), 'years_employed'] = mean_stazh[3]
df.loc[(df['years_employed'].isnull()) & (df['age_group'] == '35-45'), 'years_employed'] = mean_stazh[0]
df.loc[(df['years_employed'].isnull()) & (df['age_group'] == '45-55'), 'years_employed'] = mean_stazh[1]
df.loc[(df['years_employed'].isnull()) & (df['age_group'] == '55+'), 'years_employed'] = mean_stazh[2]
df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   years_employed    21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Мы разбили на 4 произвольные группы по возрасту с примерно одинаковым количеством записей, нашли среднее значение в каждой группе и применили к пустым с трокам. В итоге избавились от пропусков в стаже.

In [14]:
df.groupby('years_employed')['total_income'].mean()
df.groupby('income_type')['total_income'].agg(['mean','median', 'count'])

,mean,median,count
income_type,,,
безработный,131339.751676,131339.751676,2
в декрете,53829.130729,53829.130729,1
госслужащий,170898.309923,150447.935283,1312
компаньон,202417.461462,172357.950966,4577
пенсионер,137127.465690,118514.486412,3443
предприниматель,499163.144947,499163.144947,1
сотрудник,161380.260488,142594.396847,10014
студент,98201.625314,98201.625314,1


Нам необходимо заполнить пустые строки с данными о доходе, наибольшая корреляция дохода идет с типом дохода. Заполним ячейки медианным доходом по каждому типу дохода.

In [15]:
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'безработный'), 'total_income'] = 131339.75
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'в декрете'), 'total_income'] = 53829.13
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = 150447.93
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = 172357.95
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = 118514.48
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = 	499163.14
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = 142594.39
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'студент'), 'total_income'] = 98201.62
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   years_employed    21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Избавились от пропущенных значений в доходах

In [16]:
df.loc[df['children'] == -1, 'children'] /= -1
df.loc[df['children'] == 20, 'children'] /= 10

df['children'].unique()

array([1., 0., 3., 2., 4., 5.])

Исправил очевидные ошибки: заменил колиство детей "-1" на "1" и "20" на "2", Ошибки связаны с введением данных в систему. Очень смущает большое количество бездетных, что не соответствует среднему показателю по нашей стране и миру.

**Вывод**

Мы исправили данные и заполнили пропуски в столбцах со стажем, доходом,

### Замена типа данных

In [17]:
df['children'] = df['children'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   years_employed    21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


**Вывод**

В процессе преобразований данные в колонке дети поменяли свой тип, изменили обратно, так как количество детей не может быть дробным числом. Также изменили данные в колонке с доходом. Колонку со стажем оставили как есть, так как даже полгода стажа имеют значение, особенно в начале трудового пути.

### Обработка дубликатов

In [18]:
df.loc[:,'education'] = df.loc[:,'education'].str.lower()
print(df['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [19]:
df.groupby('education')['education_id'].unique()

education
высшее                 [0]
начальное              [3]
неоконченное высшее    [2]
среднее                [1]
ученая степень         [4]
Name: education_id, dtype: object

Теперь колонки 'education' и 'education_id' заполнены верно и однозначно сопоставимы

In [20]:
df['years_employed'].duplicated().sum()

2170

In [21]:
df['total_income'].duplicated().sum()

2917

In [22]:
df.duplicated(keep = False).sum()

137

Дупликаты в строке 'years_employed' только те, что мы заполнили средними, в строке 'total_income' только те, что мы заполнили средними и образовались в результате отбрасывания десятичных знаков

**Вывод**

В таблице нет дубликатов. Есть вероятность наличия дубликатов среди тех строк, где не были указаны значения стажа и дохода. Но беглый взгляд не может найти этому подтверждения. При этом, даже если все остальные значения  столбцы совпадают, это не будет явно указывать на дубликаты.

### Лемматизация

In [23]:
from pymystem3 import Mystem
m = Mystem()
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [24]:
def purpose_group(purpose):
    if 'комм' in purpose:
        return 'коммерческая недвижимость'
    if 'стро' in purpose:
        return 'строительство'       
    elif 'ремо' in purpose:
        return 'ремонт' 
    elif 'недвиж' in purpose:
        return 'недвижимость'    
    elif 'жил' in purpose:
        return 'недвижимость'
    elif 'авт' in purpose:
        return 'автомобиль'
    elif 'свад' in purpose:
        return 'свадьба' 
    elif 'образ' in purpose:
        return 'образование'    
    else:
        return 'проверь, что упустил'
    
df['purpose_group'] = df['purpose'].apply(purpose_group)
df.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group
0,1,23.101090,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,35-45,недвижимость
1,1,11.019312,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,35-45,автомобиль
2,0,15.396092,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 35,недвижимость
3,3,11.292942,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 35,образование
4,0,38.816572,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,45-55,свадьба


In [25]:

df.groupby('purpose_group')['education'].count()

purpose_group
автомобиль                   4315
коммерческая недвижимость    1315
недвижимость                 7032
образование                  4022
ремонт                        612
свадьба                      2348
строительство                1881
Name: education, dtype: int64

Мы объеденили все цели в 7 категорий.

**Вывод**

Мы объеденили все цели в 7 категорий.

### Категоризация данных

В процессе мы уже категоризовали даные по возрасту и целям получениы кредита. Для анализа необходимо сделать категоризацию по стажу и доходу.
Взглянем распределение данных

In [26]:
df.groupby('age_group')['years_employed'].agg(['mean', 'count'])

,mean,count
age_group,,
35-45,7.375558,5734
45-55,13.924247,4828
55+,31.193530,4369
до 35,4.320118,6594


Большое количество клиентов до 35 со средним стажем 4,3 года.
Можно предположить, что скорость увеличения стабильности работы клиента снижается со стажем, т.е. влияние первых лет стажа существеннее, чем последующих.
Для нашей цели решаем разбить на разные по количеству лет категории

In [27]:
def stazh_group(stazh):
    if stazh <= 2:
        return 'до 2'   
    elif 2 < stazh <= 5:
        return 'от 2 до 5'
    elif 5 < stazh <= 12:
        return 'от 5 до 12'
    elif 12 < stazh <= 40:
        return 'от 12 до 40'
    else:
        return 'от 40'
   
df['experience'] = df['years_employed'].apply(stazh_group)
df.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,experience
0,1,23.101090,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,35-45,недвижимость,от 12 до 40
1,1,11.019312,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,35-45,автомобиль,от 5 до 12
2,0,15.396092,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 35,недвижимость,от 12 до 40
3,3,11.292942,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 35,образование,от 5 до 12
4,0,38.816572,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,45-55,свадьба,от 12 до 40


In [28]:
df.groupby('experience')['income_type'].count()

experience
до 2           3852
от 12 до 40    4317
от 2 до 5      5432
от 40          2406
от 5 до 12     5518
Name: income_type, dtype: int64

Попробуем исходя из такой же концепции категоризовать клиентов по доходу

In [29]:
df.groupby('total_income')['years_employed'].agg(['mean', 'count'])

,mean,count
total_income,,
20667,40.978674,1
21205,42.175290,1
21367,9.973498,1
21695,41.036517,1
21895,39.539408,1
...,...,...
1711309,15.699184,1
1715018,12.920667,1
1726276,14.369759,1


In [30]:
def income_group(inc):
    if inc <= 75000:
        return 'до 75'
    elif 75000 < inc <= 100000:
        return 'от 75 до 100'    
    elif 100000 < inc <= 125000:
        return 'от 100 до 125' 
    elif 125000 < inc <= 150000:
        return 'от 125 до 150' 
    elif 150000 < inc <= 175000:
        return 'от 150 до 175' 
    elif 175000 < inc <= 225000:
        return 'от 175 до 225'
    else:
        return 'от 225'  
   
df['income_group'] = df['total_income'].apply(income_group)
df.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,experience,income_group
0,1,23.101090,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,35-45,недвижимость,от 12 до 40,от 225
1,1,11.019312,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,35-45,автомобиль,от 5 до 12,от 100 до 125
2,0,15.396092,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 35,недвижимость,от 12 до 40,от 125 до 150
3,3,11.292942,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 35,образование,от 5 до 12,от 225
4,0,38.816572,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,45-55,свадьба,от 12 до 40,от 150 до 175


In [31]:
df.groupby('income_group')['income_type'].count()

income_group
до 75            1865
от 100 до 125    3319
от 125 до 150    3903
от 150 до 175    3062
от 175 до 225    3004
от 225           3774
от 75 до 100     2598
Name: income_type, dtype: int64

В итоге пробовал понятные интервалы +-25 т.р чтобы получить сопоставимое предстивительство и можно было сделать понятные запоминающиеся выводы.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
df.groupby('children')['debt'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
0,14149,1063,0.075129
1,4865,445,0.091470
2,2131,202,0.094791
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Из таблицы явно следует вывод, что с появлением детей вероятность просроченной задолженности растет: 7,5% для клиентов без детей, 9,1% для клиентов с 1 ребенком дальше.

Попробуем нйти взаимосвязи с другими показателями:

In [33]:
df.groupby('children')['years_employed'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
0,14149,219100.579892,15.485234
1,4865,39103.716434,8.037763
2,2131,13527.776284,6.348088
3,330,2213.660375,6.708062
4,41,304.625110,7.429881
5,9,38.748012,4.305335


Вижу сильную корреляцию количества детей и стажа. 
Попробуем исключить влияние стажа и оценить наличие детей незевисимо от этого:

In [34]:
df.pivot_table(index='experience',columns= 'children',values='debt',aggfunc=['count','mean']).reset_index()

experience   count                                       mean  \
children                    0       1      2      3     4    5         0   
0                до 2  2290.0  1047.0  441.0   65.0   6.0  3.0  0.105240   
1         от 12 до 40  3315.0   697.0  258.0   42.0   5.0  NaN  0.054299   
2           от 2 до 5  3132.0  1490.0  700.0   96.0  12.0  2.0  0.091954   
3               от 40  2225.0   160.0   17.0    3.0   1.0  NaN  0.052135   
4          от 5 до 12  3187.0  1471.0  715.0  124.0  17.0  4.0  0.074678   

                                                       
children         1         2         3         4    5  
0         0.124164  0.115646  0.107692  0.333333  0.0  
1         0.064562  0.050388  0.047619  0.200000  NaN  
2         0.092617  0.112857  0.093750  0.000000  0.0  
3         0.050000  0.117647  0.333333  0.000000  NaN  
4         0.084296  0.079720  0.064516  0.058824  0.0

Внутри каждой группы клиентов с различным стажем мы также видим увеличение вероятности, что клиент задержит оплату с появлением детей.
Для  клиентов со стажем до 40 лет с появление 1 ребенка на обеспечении вероятность просрочки растет
Для клиентов со стажем больше 40 лет зависимость обратная
Интересно, что при поялении 2 и 3 ребенка вероятность просрочки снижается во всех группах и с 3 им ребенком становится даже ниже и очень близкой к показателям бездетных. (нельзя оценть в группе до 40, т.к. мало значений)


**Вывод**

Зависимость есть, при появлении первого ребенка вероятность возврата кредита падает, при слудующих детей возвратность растет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
df.groupby('family_status')['debt'].agg(['count','sum','mean'])

,count,sum,mean
family_status,,,
Не женат / не замужем,2813,274,0.097405
в разводе,1195,85,0.071130
вдовец / вдова,960,63,0.065625
гражданский брак,4177,388,0.092890
женат / замужем,12380,931,0.075202


Вероятность задолженности сильно выше у тех, кто никогда не был в браке:
Для тех кто никогда не был в браке 9,3% - 9,7%, для прочих 6,6% - 7,5%
Возможно на это влияют другие факторы, например стаж, который очень сильно влияет на возвратность, явно будет больше у вдовцов. Проверим распределиение стажа:


In [36]:
df.groupby('family_status')['years_employed'].agg(['count','sum','mean'])

,count,sum,mean
family_status,,,
Не женат / не замужем,2813,27315.913579,9.710598
в разводе,1195,15748.402013,13.178579
вдовец / вдова,960,25706.884108,26.778004
гражданский брак,4177,48999.678128,11.730830
женат / замужем,12380,156518.228279,12.642829


На самом деле, средний стаж вдовцов очень большой, а самый небольшой стаж у неженатых
Посмотрим внимательнее:

In [37]:
df.pivot_table(index='experience',columns= 'family_status',values='debt',aggfunc=['count','mean']).reset_index()

experience                 count                           \
family_status              Не женат / не замужем в разводе вдовец / вдова   
0                     до 2                   715       237             79   
1              от 12 до 40                   372       258            318   
2                от 2 до 5                   862       247             81   
3                    от 40                   226       140            345   
4               от 5 до 12                   638       313            137   

                                                                mean  \
family_status гражданский брак женат / замужем Не женат / не замужем   
0                          811            2010              0.132867   
1                          763            2606              0.056452   
2                         1127            3115              0.100928   
3                          414            1281              0.035398   
4                         1062            3368              0.098746   

                                                                         
family_status в разводе вдовец / вдова гражданский брак женат / замужем  
0              0.109705       0.075949         0.130703        0.098507  
1              0.065891       0.066038         0.070773        0.049117  
2              0.072874       0.086420         0.101154        0.092456  
3              0.064286       0.057971         0.048309        0.054645  
4              0.047923       0.065693         0.088512        0.073337

**Вывод**

Вероятность задолженности сильно выше у тех, кто никогда не был в браке:

Брак сейчас или в прошлом видимо делает людей более ответственными

Логика ломается для тех, у кого стаж выше 40 лет

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [38]:
df.groupby('income_group')['debt'].agg(['count','sum','mean'])

,count,sum,mean
income_group,,,
до 75,1865,136,0.072922
от 100 до 125,3319,289,0.087074
от 125 до 150,3903,335,0.085831
от 150 до 175,3062,255,0.083279
от 175 до 225,3004,241,0.080226
от 225,3774,267,0.070747
от 75 до 100,2598,218,0.083911


**Вывод**

Уровень дохода влияет на возврат задолженности вовремя не сильно, можно заметить, что клиенты в группе с наименьшим и наибольшим доходом чуть более стабильно возвращают кредиты в срок, возможно это связано с большей необходимостью планировать бюджет для первых и помощью персональных помощников для вторых.

- Как разные цели кредита влияют на его возврат в срок?

In [39]:
df.groupby('purpose_group')['debt'].agg(['count','sum','mean'])

,count,sum,mean
purpose_group,,,
автомобиль,4315,403,0.093395
коммерческая недвижимость,1315,99,0.075285
недвижимость,7032,504,0.071672
образование,4022,370,0.091994
ремонт,612,35,0.057190
свадьба,2348,186,0.079216
строительство,1881,144,0.076555


**Вывод**

Самые ответственные клиенты берут деньги на ремонт, для них вероятность стать должником 5,7%, за ними идут те, кто берут средства на покупку/строительство недвижимости как коммерческой так и личной, для них вероятность 7.2%-7,7%, за ними с небольшим отрывом желающие хорошо отметить свадьбу. Наименее ответственные покупатели авто и получающие образование, среди них доля должников 9,2% - 9,3%


## Общий вывод

Проведенный анализ показал несколько важных факторов, влияющих на погашение кредита в срок:

1. Появление первого ребенка качество погашения снижается, при появлении следующих детей начинает восстанавливаться

2. Клиенты, которые в браке или были в нем лучше погашают кредит, для категории людей с большим трудовым стажем логика обратная

3. Уровень дохода не сильно влияет на качество погашения, при этом можно выделить, что клиенты с самым высоким и самым низким дохоами в среднем стабильнее возвращают кредиты

4 Самые ответственные клиенты берут деньги на ремонт. Наименее ответственные клиенты это покупатели авто и получающие образование.